In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256, (3, 3),input_shape=[50,50,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [2]:
model.load_weights('../res/Models/modelAgeGender.h5')

In [3]:
import cv2, pickle
import dlib
import numpy as np

In [4]:
cam = cv2.VideoCapture('rtsp://192.168.1.6/live/ch00_1')#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video' '../res/test_data/videos/vid.mp4'

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

face_detector = dlib.get_frontal_face_detector()
margin = 20

In [5]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]
            s_test = test_img

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results,label,category_dict[label],acc)
            
            cv2.imwrite(f"../res/Live_Face/{category_dict[label]}({acc}%).jpg", s_test)

            cv2.rectangle(feed, (x1, y1), (x2, y2),(0, 255, 0), 2)

            cv2.putText(feed, f"{category_dict[label]}({str(acc)}%)", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    except:
        pass
    
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()